<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc_evaluation_deepeval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q langchain-groq deepeval asyncio pandas langchain langchain_google_genai vertexai instructor pydantic google-generativeai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p data
!find /content/drive/My\ Drive/tcc/resultados/ -type f ! -name '*.gsheet' -exec cp {} data/ \;

In [ ]:
import os
import glob

def get_latest_file(directory, pattern):
    file_list = glob.glob(os.path.join(directory, pattern))

    if not file_list:
        raise FileNotFoundError("No files found matching the pattern.")

    latest_file = max(file_list, key=os.path.getmtime)

    return latest_file

    # return open(latest_file, 'r')



In [ ]:
directory = 'data/'
pattern = 'QA_ppc2024*.csv'


latest_file = get_latest_file(directory, pattern)
print(latest_file)

data/QA_ppc2024_08_10_2024_10-40-55.csv


In [ ]:
import pandas as pd
df = pd.read_csv(latest_file)

In [ ]:
from google.colab import userdata

class Settings:
  GROQ_API_KEY = userdata.get('GROQ_API_KEY')
  GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
  MISTRAL_AI_KEY = userdata.get('MISTRAL_AI_KEY')
  GROQ_API_KEY2 = userdata.get('GROQ_API_KEY2')



settings = Settings()

In [ ]:
import json
import random
from typing import get_origin, get_args, List, Dict, Union
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM
from pydantic import BaseModel, ValidationError
from time import sleep

class CustomGroqModel(DeepEvalBaseLLM):
    def __init__(self, model_id="llama-3.2-90b-text-preview"):
        self.MAX_RETRIES = 3
        self.api_keys = [settings.GROQ_API_KEY, settings.GROQ_API_KEY2]
        self.model_id = model_id

    def load_model(self, api_key):
        return ChatGroq(
            model=self.model_id,
            temperature=0.0,
            max_retries=4,
            api_key=api_key
        )

    def get_default_value(self, field_type):
        # Handle Union types
        if get_origin(field_type) is Union:
            args = get_args(field_type)
            # Try to find a non-None type in the Union
            for arg in args:
                if arg is not type(None):
                    return self.get_default_value(arg)
            return None

        # Handle List types
        if get_origin(field_type) is list:
            return []

        # Handle Dict types
        if get_origin(field_type) is dict:
            return {}

        # Handle basic types
        type_defaults = {
            str: "",
            int: 0,
            float: 0.0,
            bool: False,
            dict: {},
            list: [],
        }

        # If it's a BaseModel, recursively generate null schema
        if isinstance(field_type, type) and issubclass(field_type, BaseModel):
            return self.generate_null_schema(field_type)

        return type_defaults.get(field_type, None)

    def generate_null_schema(self, schema: BaseModel) -> BaseModel:
        try:
            default_values = {}
            for field_name, field in schema.__fields__.items():
                field_type = field.outer_type_
                default_values[field_name] = self.get_default_value(field_type)

            return schema.parse_obj(default_values)
        except ValidationError as e:
            print(f"Schema validation error: {e}")
            return None

    def generate(self, prompt: str, schema: BaseModel, attempt=0) -> BaseModel:
        selected_api_key = random.choice(self.api_keys)
        chat_model = self.load_model(selected_api_key)

        if schema:
            structured_llm = chat_model.with_structured_output(schema)
        else:
            print(prompt)
            return chat_model.invoke(prompt).content

        try:
            response = structured_llm.invoke(prompt)
            print(response)
            return response
        except Exception as e:
            print(f"Error generating response: {str(e)}, Attempt {attempt + 1}")

            if attempt < self.MAX_RETRIES:
                sleep(1)  # Add a small delay before retrying
                return self.generate(prompt, schema, attempt + 1)
            else:
                print(f"Max retries exceeded. Returning null schema.")
                return self.generate_null_schema(schema)

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return self.model_id

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from deepeval.models.base_model import DeepEvalBaseLLM

class GoogleGenerativeAI(DeepEvalBaseLLM):
    """Class to implement Google Generative AI for DeepEval"""

    def __init__(self, model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()

        response = chat_model.invoke(prompt).content
        print(response)
        return response

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Google Generative AI Model"

gemini_llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash-latest",
    convert_system_message_to_human=True,
    google_api_key=userdata.get('GEMINI_API_KEY'),
    temperature=0.3,
    top_k=1,
    top_p=0.9,
    max_output_tokens=8192,
    verbose=True
)

gemini = GoogleGenerativeAI(model=gemini_llm)

In [ ]:
from pydantic import BaseModel
import google.generativeai as genai
import instructor

class CustomGeminiFlash(DeepEvalBaseLLM):
    def __init__(self):
        genai.configure(api_key = settings.GEMINI_API_KEY)
        self.model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        client = self.load_model()
        instructor_client = instructor.from_gemini(
            client=client,
            mode=instructor.Mode.GEMINI_JSON,
        )
        resp = instructor_client.messages.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            response_model=schema,
        )
        print(resp)
        return resp

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return "Gemini 1.5 Flash"

In [ ]:
from deepeval.test_case import LLMTestCase

def context_handler(contexts):
    context_list = [context for context in contexts.split('\n<|FIM_DO_CHUNK|>\n') if context]
    if len(context_list) >= 6:
        # Concatenate pairs into single strings
        return [' '.join(context_list[i:i+2]) for i in range(0, len(context_list), 2)]
    return context_list

test_cases = [
    LLMTestCase(
        input=row['Pergunta'],
        actual_output=row['RespostaGerada'],
        expected_output=row['RespostaIdeal'],
        retrieval_context=context_handler(row['Contexto'])
    )
    for _, row in df.iterrows()
]


In [ ]:
test_cases = [item for index, item in enumerate(test_cases) if index not in [49, 50]]

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric, ContextualRecallMetric, ContextualPrecisionMetric, FaithfulnessMetric, GEval, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCaseParams

def generate_deepeval_metrics(model):
    # Define correctness metric with model set
    correctness_metric = GEval(
        name="Correctness",
        criteria="Determine whether the actual output is factually correct based on the expected output.",
        evaluation_steps=[
            "Check whether the facts in 'actual output' contradict any facts in 'expected output'",
            "You should also heavily penalize omission of detail",
            "Vague language, or contradicting OPINIONS, are OK"
        ],
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
        model=model
    )

    answer_relevancy_metric = AnswerRelevancyMetric(
        model=model,
        threshold=0.5
    )

    contextual_recall_metric = ContextualRecallMetric(
        threshold=0.25,
        model=model,
    )

    contextual_precision_metric = ContextualPrecisionMetric(
        threshold=0.5,
        model=model,
    )

    contextual_relevancy = ContextualRelevancyMetric(
        model = model,
        threshold=0.5
    )

    faithfulness_metric = FaithfulnessMetric(
        model=model,
        threshold=0.5
    )

    # Return the metrics in a list
    return [
        correctness_metric,
        answer_relevancy_metric,
        contextual_recall_metric,
        # contextual_relevancy,
        faithfulness_metric
    ]


In [ ]:
import json
import os


def parse_test_result_to_dict(test_result):
    parsed_results = []

    for input_key, test_result_list in test_result.items():
        for test_result_obj in test_result_list:
            result_dict = {
                'input': input_key,
                'success': test_result_obj.success,
                'conversational': test_result_obj.conversational,
                'multimodal': test_result_obj.multimodal,
                'actual_output': test_result_obj.actual_output,
                'expected_output': test_result_obj.expected_output,
                'context': test_result_obj.context,
                'retrieval_context': test_result_obj.retrieval_context
            }

            metrics_list = []
            for metric in test_result_obj.metrics_data:
                metric_dict = {
                    'name': metric.name,
                    'threshold': metric.threshold,
                    'success': metric.success,
                    'score': metric.score,
                    'reason': metric.reason,
                    'strict_mode': metric.strict_mode,
                    'evaluation_model': metric.evaluation_model,
                    'error': metric.error,
                    'evaluation_cost': metric.evaluation_cost,
                    'verbose_logs': metric.verbose_logs
                }
                metrics_list.append(metric_dict)

            result_dict['metrics'] = metrics_list
            parsed_results.append(result_dict)

    return parsed_results


def save_dict_to_json_on_drive(data_dict, drive_folder_path, drive_file_name):
    os.makedirs(drive_folder_path, exist_ok=True)
    drive_path = f'{drive_folder_path}/{os.path.basename(drive_file_name)}'
    with open(drive_path, 'w', encoding='utf-8') as json_file:
        json.dump(data_dict, json_file, ensure_ascii=False)
    print(f'File saved to: {drive_path}')

def transform_and_save_results(results, identifier = 'Groq_'):
  # parsed_results = parse_test_result_to_dict(results)
  save_dict_to_json_on_drive(results, '/content/drive/MyDrive/tcc/resultados/validados/deepeval', identifier + latest_file.split('/')[1].replace('.csv','.json'))

In [ ]:
# from deepeval import evaluate
# from deepeval import assert_test
# from deepeval.dataset import EvaluationDataset

# custom_groq = CustomGroqModel()

# groq_metrics = generate_deepeval_metrics(custom_groq)
# # groq_results = dataset.evaluate(groq_metrics)
# groq_results = {}
# for test_case in test_cases:
#     try:
#       test_case_result = {}

#       print(f"\nPergunta: {test_case.input}")
#       measurment = EvaluationDataset(test_cases=[test_case]).evaluate(groq_metrics)

#       # for metric in groq_metrics:
#       #     metric_name = metric.__class__.__name__  # Nome da métrica
#       #     measurment = EvaluationDataset(test_cases=[test_case]).evaluate([metric])
#       #     test_case_result[metric_name] = measurment
#       #     sleep(10)
#       print('Measurment: \n', measurment)

#       groq_results[test_case.input] = measurment
#       sleep(30)
#     except Exception:
#       continue


In [ ]:
from deepeval import evaluate
from deepeval.dataset import EvaluationDataset
from typing import List, Dict, Any, Optional
from time import sleep

def create_metric_result(measurement) -> Dict:
    metrics_data = measurement.metrics_data[0]

    return {
        'name': metrics_data.name,
        'threshold': metrics_data.threshold,
        'success': metrics_data.success,
        'score': metrics_data.score,
        'reason': metrics_data.reason,
        'strict_mode': metrics_data.strict_mode,
        'evaluation_model': metrics_data.evaluation_model,
        'error': metrics_data.error,
        'evaluation_cost': metrics_data.evaluation_cost,
        'verbose_logs': metrics_data.verbose_logs
    }

def evaluate_test_case(test_case, metrics, add_sleep: bool = False) -> Dict:
    dataset = EvaluationDataset(test_cases=[test_case])
    metric_results = []
    overall_success = True

    for metric in metrics:
        try:
            measurement = dataset.evaluate([metric])[0]
            metric_result = create_metric_result(measurement)

            if not metric_result['success']:
                overall_success = False

            metric_results.append(metric_result)

            if add_sleep:
                sleep(15)
        except Exception as e:
            print(f"Error evaluating metric {metric.name}: {str(e)}")
            continue

    return {
        'input': test_case.input,
        'success': overall_success,
        'actual_output': test_case.actual_output,
        'expected_output': test_case.expected_output,
        'context': test_case.context,
        'retrieval_context': test_case.retrieval_context,
        'metrics': metric_results
    }

def evaluate_model(model, test_cases: List, add_sleep: bool = False) -> List[Dict]:

    model_metrics = generate_deepeval_metrics(model)
    results = []

    for test_case in test_cases:
        try:
            evaluation_result = evaluate_test_case(test_case, model_metrics, add_sleep)
            results.append(evaluation_result)

            if add_sleep:
                sleep(20)
        except Exception as e:
            print(f"Failed to evaluate test case: {str(e)}")
            continue

    return results

In [ ]:
# custom_groq = CustomGroqModel('llama-3.1-70b-versatile')
# # generate_deepeval_metrics(custom_groq)[0].name
# groq_results = evaluate_model(custom_groq, test_cases[41:42])
# transform_and_save_results(groq_results, 'Groq_teste')

In [ ]:
!pip install -qU langchain_mistralai

In [ ]:
# from langchain_mistralai import ChatMistralAI


# mixtral_large = ChatMistralAI(
#     model="mistral-large-latest",
#     temperature=0,
#     max_retries=4,
#     api_key = settings.MISTRAL_AI_KEY,
#     # other params...
# )

# mixtral_large.invoke('Oi Mixtral, posso lhe utilizar para avaliar output de sistemas RAG?')

In [ ]:
import json
import random
from typing import get_origin, get_args, List, Dict, Union
from langchain_groq import ChatGroq
from langchain_mistralai import ChatMistralAI
from deepeval.models.base_model import DeepEvalBaseLLM
from pydantic import BaseModel, ValidationError
from time import sleep

class LangchainModel(DeepEvalBaseLLM):
    def __init__(self, provider="Groq", model_id="llama-3.2-90b-text-preview"):
        self.MAX_RETRIES = 3
        self.model_id = model_id
        self.provider = provider
        self.api_keys = (
            [settings.GROQ_API_KEY, settings.GROQ_API_KEY2] if provider == "Groq"
            else [settings.MISTRAL_AI_KEY]
        )
        self.model = self.load_model()

    def load_model(self):
        if self.provider == "Groq":
            selected_api_key = random.choice(self.api_keys)
            return ChatGroq(model=self.model_id, temperature=0.0, max_retries=4, api_key=selected_api_key)
        elif self.provider == "Mistral":
            return ChatMistralAI(model=self.model_id, temperature=0, max_retries=4, api_key=settings.MISTRAL_AI_KEY)

    def get_default_value(self, field_type):
        if get_origin(field_type) is Union:
            for arg in get_args(field_type):
                if arg is not type(None):
                    return self.get_default_value(arg)
            return None

        if get_origin(field_type) is list:
            return []

        if get_origin(field_type) is dict:
            return {}

        type_defaults = {str: "", int: 0, float: 0.0, bool: False, dict: {}, list: []}

        if isinstance(field_type, type) and issubclass(field_type, BaseModel):
            return self.generate_null_schema(field_type)

        return type_defaults.get(field_type, None)

    def generate_null_schema(self, schema: BaseModel) -> BaseModel:
        try:
            default_values = {field_name: self.get_default_value(field.outer_type_) for field_name, field in schema.__fields__.items()}
            return schema.parse_obj(default_values)
        except ValidationError as e:
            print(f"Schema validation error: {e}")
            return None

    def generate(self, prompt: str, schema: BaseModel = None, attempt=0) -> BaseModel:
        chat_model = self.load_model()

        if schema:
            structured_llm = chat_model.with_structured_output(schema)
        else:
            return chat_model.invoke(prompt).content

        try:
            response = structured_llm.invoke(prompt)
            return response
        except Exception as e:
            if attempt < self.MAX_RETRIES:
                sleep(1)
                return self.generate(prompt, schema, attempt + 1)
            return self.generate_null_schema(schema)

    async def a_generate(self, prompt: str, schema: BaseModel = None) -> BaseModel:
        return self.generate(prompt, schema)


    def get_model_name(self):
        return self.model_id


In [ ]:
test_cases[51]

LLMTestCase(input='Como é a ficha de avaliação do TCC I:', actual_output='A ficha de avaliação do TCC I, de acordo com o PPC, é apresentada no trecho fornecido e contém os seguintes itens:\n\n- Relevância do trabalho/tema para a área específica e para a carreira profissional (máximo 1,0)\n- Argumentação e sequenciamento de ideias (máximo 1,0)\n- Objetivos gerais e específicos condizentes (máximo 1,0)\n- Explicitação dos procedimentos metodológicos e propriedade de análise (máximo 1,0)\n- Rigor conceitual, estrutura lógica e consistência dos argumentos (máximo 2,0)\n- Clareza e correção linguística do texto (máximo 2,0)\n- Pertinência e uso adequado de contextualização bibliográfica (máximo 2,0)\n\nA nota final do TCC I será calculada com base na média aritmética das notas proferidas pelos membros da banca avaliadora, conforme formulário de avaliação presente no Anexo I do regulamento do TCC.', expected_output='A ficha de avaliação do TCC I está dividida em duas partes: avaliação da esc

In [ ]:
mixtral = LangchainModel('Mistral', 'mistral-large-latest')

In [ ]:
mixtral_metrics = generate_deepeval_metrics(mixtral)
mixtral_results = evaluate_model(mixtral, test_cases, False)
transform_and_save_results(mixtral_results, 'MixtralLarge_')

✨ You're running DeepEval's latest Correctness (GEval) Metric! (using mistral-large-latest, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:14, 14.30s/test case]



Metrics Summary

  - ✅ Correctness (GEval) (score: 1.0, threshold: 0.5, strict: False, evaluation model: mistral-large-latest, reason: The actual output provides a detailed and accurate description of the syllabus for Introdução à Computação, including specific topics and areas of focus. It does not contradict any facts in the expected output and provides comprehensive information., error: None)

For test case:

  - input: Qual é a ementa de Introdução à Computação?
  - actual output: A ementa do componente curricular Introdução à Computação no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS é a seguinte:

"Introdução ao Curso de Ciência da Computação e à Universidade Federal da Fronteira Sul: principais estruturas e regulamentos. Introdução à área da Computação: inserção e desenvolvimento profissional, carreira acadêmica e técnica. Conhecimentos e comandos básicos em sistemas operacionais abertos. Fundamentos básicos da Computação: história, principais componentes d

✓ Tests finished 🎉! Run 'deepeval login' to view evaluation results on Confident AI. 
‼️  NOTE: You can also run evaluations on ALL of deepeval's metrics directly on Confident AI instead.

...

None


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:28, 28.87s/test case]



Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.5, strict: False, evaluation model: mistral-large-latest, reason: The score is 1.00 because there are no contradictions! Great job!, error: None)

For test case:

  - input: O curso oferece vagas para transferência externa ou interna?
  - actual output: Sim, o curso oferece vagas para transferência externa e interna.

A transferência interna é mencionada no item 1.16 do PPC, onde se afirma que o acesso aos cursos de graduação da UFFS pode ocorrer por meio de diferentes formas de ingresso, incluindo a transferência interna. Além disso, é especificado que a transferência interna está vedada no semestre de ingresso ou de retorno para a UFFS.

Já a transferência externa é mencionada no item 195 do PPC, sob a rubrica "Transferência externa", que descreve a concessão de vaga a discentes regularmente matriculados em outras instituições de ensino superior, nacional ou estrangeira, para prosseguimento de seus estudos na UFFS.
  - e

✓ Tests finished 🎉! Run 'deepeval login' to view evaluation results on Confident AI. 
‼️  NOTE: You can also run evaluations on ALL of deepeval's metrics directly on Confident AI instead.

File saved to: /content/drive/MyDrive/tcc/resultados/validados/deepeval/MixtralLarge_QA_ppc2024_08_10_2024_10-40-55.json


In [ ]:
for res in mixtral_results:
  if not res['success']:
    print(res)

{'input': 'Qual é a ementa de Engenharia de Software?', 'success': False, 'actual_output': 'A ementa da disciplina de Engenharia de Software consiste em:\n\n- Engenharia de software\n- Processo e ciclo de vida de desenvolvimento de software\n- Comparação entre processo tradicional e ágil\n- Métodos de análise e de projeto de software\n- Engenharia de requisitos\n- Gerenciamento de configuração de software e ferramentas para controle de versão\n- Verificação, validação e teste\n- Utilização da UML para projetos de software\n- Noções de usabilidade e critérios ergonômicos.\n\nEsse conteúdo está descrito na tabela do PPC relacionada à componente curricular "Engenharia de Software Draft".', 'expected_output': 'Engenharia de software. Processo e ciclo de vida de desenvolvimento de software. Comparação entre processo tradicional e ágil. Métodos de análise e de projeto de software. Engenharia de requisitos. Gerenciamento de configuração de software e ferramentas para controle de versão. Verif